In [ ]:
!pip install rdflib

In [ ]:
!pip install owlready2

In [3]:
import os
from rdflib import Graph,Literal, RDF, URIRef
from rdflib.namespace import FOAF, XSD
from rdflib.namespace import  RDF, RDFS

cwd=os.getcwd()

### Fetch data from XML

In [4]:
import xml.etree.ElementTree as ET

tree = ET.parse('myxml.xml')
root = tree.getroot()

waiters = []
AllWaiters = root.findall("Branch/Staff/Waiter")
for w in AllWaiters:
    wId = w.get('Id')
    name = w.find('Name')
    salary = w.find('Salary')
    contact = w.find('Contact')
    phn = contact[0]
    email = contact[1]
    waiters.append([wId,name.text,salary.text,phn.text,email.text])

orders = []
Allorders = root.findall("Branch/Customer/Order")
for w in Allorders:
    #waiters.append(w)
    oId = w.get('OrderNo')
    Date = w.find('Date')
    details = "Date = "+Date[0].text+" "+Date[1].text+" "+Date[2].text+" | "
    amount = w.find('Amount')
    details = details+"Amount = "+amount.text+" | Items ="
    items = w.findall('OrderItem')
    ordItems = set()
    for item in items:
      it = item.get('itemId')
      ordItems.add(it)
      details = details + ' '+it
    wait = w.find('ServedBy')
    servBy = wait.get('waiterId')
    orders.append([oId,servBy,details,ordItems])

In [ ]:
# print(*waiters, sep = "\n")

In [ ]:
# print(*orders, sep = "\n")

In [5]:
Foods = {}
Beverages = {}
AllStarter = root.findall("Branch/MenuCard/Starter/Item")
AllMainCourse = root.findall("Branch/MenuCard/MainCourse/Item")
AllDessert = root.findall("Branch/MenuCard/Dessert/Item")  
for w in AllStarter:
  id=w.get('Id')
  if(w[0].tag == "Food"):
    Foods[id]=w[0].text
  elif(w[0].tag == "Beverage"):
    Beverages[id]=w[0].text
for w in AllMainCourse:
  id=w.get('Id')
  if(w[0].tag == "Food"):
    Foods[id]=w[0].text
  elif(w[0].tag == "Beverage"):
    Beverages[id]=w[0].text
for w in AllDessert:
  id=w.get('Id')
  if(w[0].tag == "Food"):
    Foods[id]=w[0].text
  elif(w[0].tag == "Beverage"):
    Beverages[id]=w[0].text

In [ ]:
# f = Foods.values()
# for i in f:
#   print(f'{i} {type(i)}')

# f = Beverages.values()
# for i in f:
#   print(f'{i} {type(i)}')

In [6]:
g=Graph()
filename = cwd+"/RestaurantOntologyRdf.owl"
g.load(filename, format='xml')

def isAlreadyDefined(subs):
    for s in g.subjects():
        if(subs in str(s)):
            return True
    return False

myNamespace="http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto"
namedIndividual = URIRef('http://www.w3.org/2002/07/owl#NamedIndividual')
rdftype = URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type")

In [7]:
triplets=[]
obj_properties=[]  
att_properties=[]  

arc_class=str(myNamespace)+"#Waiter"
for indv in waiters:
  individualName=str(myNamespace)+"#"+str(indv[1]).replace(' ','_')
  arc_individual = URIRef(individualName)
  if(isAlreadyDefined(individualName)==False): 
    triplets.append((arc_individual,RDF.type, URIRef(arc_class)))
    triplets.append((arc_individual,RDF.type, URIRef(namedIndividual)))

    subject=arc_individual
    #pred= URIRef(str(myNamespace)+"#id")
    literal=indv[0]
    att_properties.append( (subject, URIRef(str(myNamespace)+"#id")  , Literal(literal,datatype=XSD.string)) )
    att_properties.append( (subject, URIRef(str(myNamespace)+"#name"), Literal(indv[1],datatype=XSD.string)) )
    att_properties.append( (subject, URIRef(str(myNamespace)+"#salary"), Literal(indv[2],datatype=XSD.string)) )
    

arc_class=str(myNamespace)+"#Order"
for indv in orders:
  individualName=str(myNamespace)+"#"+str(indv[0])
  arc_individual = URIRef(individualName)
  if(isAlreadyDefined(individualName)==False): 
    triplets.append((arc_individual,RDF.type, URIRef(arc_class)))
    triplets.append((arc_individual,RDF.type, URIRef(namedIndividual)))

    subject=arc_individual
    att_properties.append( (subject, URIRef(str(myNamespace)+"#orderDetails"), Literal(indv[2],datatype=XSD.string)) )

arc_class=str(myNamespace)+"#Food"
foodList = Foods.values()
for indv in foodList:
  individualName=str(myNamespace)+"#"+str(indv).replace(' ','_')
  # print(individualName)
  arc_individual = URIRef(individualName)
  if(isAlreadyDefined(individualName)==False): 
    triplets.append((arc_individual,RDF.type, URIRef(arc_class)))
    triplets.append((arc_individual,RDF.type, URIRef(namedIndividual)))

arc_class=str(myNamespace)+"#Beverage"
BeverageList = Beverages.values()
for indv in BeverageList:
  individualName=str(myNamespace)+"#"+str(indv).replace(' ','_')
  # print(individualName)
  arc_individual = URIRef(individualName)
  if(isAlreadyDefined(individualName)==False): 
    triplets.append((arc_individual,RDF.type, URIRef(arc_class)))
    triplets.append((arc_individual,RDF.type, URIRef(namedIndividual)))

In [ ]:
# print(att_properties[2])

(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rakul'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#salary'), rdflib.term.Literal('18000', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))


In [ ]:
# print(att_properties[51])

(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#D100001'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#orderDetails'), rdflib.term.Literal('Date = 20 March 2022 | Amount = 252 | Items = I103 I120 I111', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))


### Add object properties "takes"
#### Waiter takes orders
#### Use attribute waiterId ServedBy provided in the XML to make the mapping

In [8]:
for waiter in waiters:
    individualName=str(myNamespace)+"#"+str(waiter[1]).replace(' ','_')  
    sub = URIRef(individualName)         
    pred=URIRef((myNamespace)+"#takes")

    for ord in orders: 
        if(ord[1]==waiter[0]):
            individualName=str(myNamespace)+"#"+str(ord[0]).replace(' ','_') 
            obj = URIRef(individualName)
            obj_properties.append((sub,pred,obj))   

### Add object properties "contains"
#### Order contains Food
#### Use attribute waiterId ServedBy provided in the XML to make the mapping

In [9]:
for order in orders:
  individualName=str(myNamespace)+"#"+str(order[0]).replace(' ','_')
  sub = URIRef(individualName)         
  pred=URIRef((myNamespace)+"#contains")

  # print(order[3])
  for item in order[3]:
    if (Foods.get(item)!=None):
      nn = Foods.get(item)
      individualName=str(myNamespace)+"#"+str(nn).replace(' ','_')
      obj = URIRef(individualName)
      obj_properties.append((sub,pred,obj))
    elif(Beverages.get(item)!=None):
      nn = Beverages.get(item)
      individualName=str(myNamespace)+"#"+str(nn).replace(' ','_')
      obj = URIRef(individualName)
      obj_properties.append((sub,pred,obj))

In [ ]:
# print(*obj_properties, sep = "\n")

###Print triplets

In [10]:
for triplet in triplets:
  print(triplet)


(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rakul'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Waiter'))
(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rakul'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#NamedIndividual'))
(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rahul_Sharma'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Waiter'))
(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rahul_Sharma'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#NamedIndividu

In [11]:
for o in obj_properties:
  print(o)

(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rakul'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#takes'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#D100001'))
(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rakul'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#takes'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#D100030'))
(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rakul'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#takes'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#D100032'))
(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rahul_Sharma'), rdflib.term.URIRef('http://www.semanticweb.

In [12]:
for a in att_properties:
  print(a)

(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rakul'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#id'), rdflib.term.Literal('S011', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rakul'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#name'), rdflib.term.Literal('Rakul', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rakul'), rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#salary'), rdflib.term.Literal('18000', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Rahul_Sharma'), rdflib.term.URIRef('http://ww

In [10]:
print("Total Added Triples = ",int(len(triplets))+int(len(obj_properties))+int(len(att_properties)))
print(int(len(triplets)))
print(int(len(obj_properties)))
print(int(len(att_properties)))

Total Added Triples =  389
156
147
86


###Add tiplets

In [11]:
for i in triplets:
    g.add(i)

for i in obj_properties:
    g.add(i)

for i in att_properties:
    g.add(i)

In [12]:
count= 0
for s,p,o in g:
  count +=1
print(count)

1425


In [13]:
g.serialize(destination="draft1.owl",format='xml')

<Graph identifier=N6da0e517f6024488a53abb0a3b52f56e (<class 'rdflib.graph.Graph'>)>

In [14]:
from owlready2 import *

onto_path.append(cwd)
onto = get_ontology("file://draft1.owl").load()

In [18]:
print(type(onto))

<class 'owlready2.namespace.Ontology'>


In [15]:
with onto: sync_reasoner()
onto.save("RestaurantOntologyRdf_syn.owl")

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/lib/python3.7/dist-packages/owlready2/hermit:/usr/local/lib/python3.7/dist-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpz5kq2n9b
* Owlready2 * HermiT took 4.445778131484985 seconds
* Owlready * Reparenting draft1.managesActively: {owl.ObjectProperty, draft1.manages} => {draft1.manages}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [16]:
import rdflib
g2=rdflib.Graph()
fl = "RestaurantOntologyRdf_syn.owl"
g2.load(fl, format='xml')

<Graph identifier=N6f72950772404e648307e11bbaaba737 (<class 'rdflib.graph.Graph'>)>

In [17]:
c=0
for t in g2.triples((None,None,None)):
  c+=1
print(c)

1435


In [18]:
from rdflib.compare import to_isomorphic, graph_diff
iso1 = to_isomorphic(g)
iso2 = to_isomorphic(g2)

in_both, in_first, in_second = graph_diff(iso1,iso2)

def dump_nt_sorted(g):
    for l in sorted(g.serialize(format='nt').splitlines()):
        if(l) : print(l)

dump_nt_sorted(in_second)

<http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#BillingCounter> <http://www.w3.org/2000/01/rdf-schema#subClassOf> <http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Area> .
<http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Dessert> <http://www.w3.org/2000/01/rdf-schema#subClassOf> <http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Dish> .
<http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Kitchen> <http://www.w3.org/2000/01/rdf-schema#subClassOf> <http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Area> .
<http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#MainCourse> <http://www.w3.org/2000/01/rdf-schema#subClassOf> <http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#Dish> .
<http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#PartyHall> <http://www.w3.org/2000/01/rdf-schema#subClassOf> <http://www.semanticweb.org/ajay/ontologies/2022/3/RestaurantOnto#A